In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import ImageFile
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

In [4]:
train_datagen = ImageDataGenerator( rescale = 1./255, 
                                   shear_range = 0.2, 
                                   zoom_range = [.99,1.01], 
                                   brightness_range = [0.8,1.2], 
                                   data_format = "channels_last", 
                                   fill_mode ="constant", 
                                   horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/IBM/dataset/train',
target_size = (180, 180), batch_size = 64,
class_mode = 'categorical')
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/IBM/dataset/test',
target_size = (180, 180), batch_size = 64,
class_mode = 'categorical')

Found 1500 images belonging to 5 classes.
Found 300 images belonging to 5 classes.


In [6]:
IMAGE_SIZE =[180,180]
VGG19 = VGG19(input_shape=IMAGE_SIZE + [3], weights='imagenet',include_top=False)

80134624/80134624 [==============================] - 0s 0us/step


In [7]:
for layer in VGG19.layers: 
  layer.trainable = False
x = Flatten() (VGG19.output)

In [8]:
prediction = Dense(5, activation='softmax')(x)
model = Model(inputs=VGG19.input, outputs=prediction)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 180, 180, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 180, 180, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 180, 180, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 90, 90, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 90, 90, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 90, 90, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 45, 45, 128)       0     

In [9]:
model.compile(
loss='categorical_crossentropy',
optimizer='adam',
metrics=['accuracy'] )

In [10]:
r = model.fit(
training_set,
validation_data = test_set,
epochs = 50,
steps_per_epoch=len(training_set),
validation_steps = len(test_set)
)

Epoch 1/50
24/24 [==============================] - 483s 20s/step - loss: 1.2283 - accuracy: 0.5360 - val_loss: 0.7375 - val_accuracy: 0.7167
Epoch 2/50
24/24 [==============================] - 30s 1s/step - loss: 0.6143 - accuracy: 0.7813 - val_loss: 0.4687 - val_accuracy: 0.8167
Epoch 3/50
24/24 [==============================] - 30s 1s/step - loss: 0.4451 - accuracy: 0.8700 - val_loss: 0.3806 - val_accuracy: 0.8767
Epoch 4/50
24/24 [==============================] - 29s 1s/step - loss: 0.3700 - accuracy: 0.8940 - val_loss: 0.2831 - val_accuracy: 0.9233
Epoch 5/50
24/24 [==============================] - 29s 1s/step - loss: 0.3335 - accuracy: 0.9007 - val_loss: 0.2344 - val_accuracy: 0.9500
Epoch 6/50
24/24 [==============================] - 31s 1s/step - loss: 0.2718 - accuracy: 0.9213 - val_loss: 0.2224 - val_accuracy: 0.9533
Epoch 7/50
24/24 [==============================] - 30s 1s/step - loss: 0.2586 - accuracy: 0.9213 - val_loss: 0.1884 - val_accuracy: 0.9600
Epoch 8/50
24/24 [

In [11]:
loss, accuracy = model.evaluate(test_set,
steps=11,
verbose=2,
use_multiprocessing=True,
workers=2)
print(f'Model performance on test images:\nAccuracy = {accuracy}\nLoss = {loss}')

11/11 - 8s - loss: 0.0220 - accuracy: 0.9970 - 8s/epoch - 750ms/step
Model performance on test images:
Accuracy = 0.9969879388809204
Loss = 0.021998772397637367


In [12]:
model.save('wcv.h5')

In [18]:
model = load_model("/content/wcv.h5")

img = image.load_img(r"/content/drive/MyDrive/IBM/dataset/train/sunrise/sunrise1.jpg",target_size= (180,180))
x = image.img_to_array(img)
x = np.expand_dims(x,axis = 0)
preds=model.predict(x)
pred=np.argmax(preds,axis=1)
index=['cloudy','foggy','rainy','shine','sunrise']
result=str(index[pred[0]])
result

1/1 [==============================] - 0s 151ms/step


'sunrise'